In [1]:
import pandas as pd
import requests
import numpy as np
import os
import json
from dotenv import load_dotenv
from dateutil import parser
import datetime as dt
import spacy



In [2]:
load_dotenv()
bearer_token = os.getenv("BEARER_TOKEN")


In [3]:
search_url = "https://api.twitter.com/2/tweets/search/recent"


In [4]:
query_params = {
    'query': 'lang=en',
    'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
    'tweet.fields': 'author_id,created_at,geo,id,lang,possibly_sensitive,source,text,withheld',
    'user.fields': 'id,location,name,protected,username,verified,withheld',
    'max_results': 100}
# query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}

In [5]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [6]:
def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [9]:
def main():
    # App se conecta al endpoint de Twitter
    json_response = connect_to_endpoint(search_url, query_params)

    # El reusltado de la consulta se transforma en un Data Frame
    df = pd.DataFrame.from_dict(json_response['data'])

    # Se define la variabl time para establecer el timeline de Twitts
    df['time'] = pd.to_datetime(df['created_at'])
        
    # Lista de palabras a exlcuir
    nlp = spacy.load('en_core_web_sm')
    excluded_tags = {"SYM", "PUNCT", "PART", "AUX", "CCONJ", "PRON", "NOUN", "ADV", "ADP", "PROPN"}
    banned = ["the", "a"]
    
    #Almacenamos las palabras en una lista, excepcionando las palabras a excluir
    single_text = []
    list_of_text = df['text'].str.split()

    for text in list_of_text:
        doc = nlp(' '.join(text))
        for token in doc:
            if (token.pos_ not in excluded_tags) and (token.text.lower() not in banned)  :
                single_text.append(token.text.lower())
    

    top_repeated = pd.Series(single_text).value_counts().to_dict()
        

In [10]:
if __name__ == "__main__":
    main()

200


/Users/javieraquique/Applications/tweetsApp/tweetsApp/.env/lib/python3.9/site-packages/spacy/util.py:885: UserWarning: [W094] Model 'en_core_web_sm' (2.2.0) specifies an under-constrained spaCy version requirement: >=2.2.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.4.2,<3.5.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /Users/javieraquique/Applications/tweetsApp/tweetsApp/.env/lib/python3.9/site-packages/en_core_web_sm/en_core_web_sm-2.2.0/config.cfg